In [ ]:
import PyPDF2
import json
import pandas as pd
import re
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            extracted = page.extract_text()
            if extracted:
                text += extracted + "\n"
    return text.strip()

# Path to PDF (Replace with actual path)
pdf_path = "test3.pdf"
extracted_text = extract_text_from_pdf(pdf_path)

# LLM setup
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",  # Replace securely
    model="llama-3.1-8b-instant"
)

# Prompt template for structured JSON output
prompt_extract = PromptTemplate.from_template(
    """
    You are an expert in financial statement table extraction. 
    Extract **only table data** from the provided text and return **strict JSON output**.

    ### Extracted Text:
    {extracted_text}

    ### OUTPUT REQUIREMENTS:
    - Extract **only valid table structures** (ignore non-table text).
    - Preserve column headers and data accurately.
    - Ensure **headers and rows are properly separated**.
    - Avoid mixing table headers with data.
    - Do **not truncate** the JSON output.

    ### STRICT JSON FORMAT:
    {{
        "tables": [
            {{
                "table_id": 1,
                "headers": ["Date", "Transaction Type", "Debit/Credit", "Amount", "Description"],
                "rows": [
                    ["01-APR-2022", "Opening Balance", "Dr", "30,63,234.66", ""],
                    ["04-APR-2022", "Transfer", "Cr", "25,000.00", "BY 06971000010040"],
                    ["04-APR-2022", "Cash", "Cr", "40,000.00", "By Cash"]
                ]
            }}
        ]
    }}
    """
)

# Running the LLM extraction
chain_extract = prompt_extract | llm
response = chain_extract.invoke(input={'extracted_text': extracted_text})
llm_response = response.content

# Fixing JSON formatting issue
cleaned_json = re.sub(r"```json\n|\n```", "", llm_response).strip()

try:
    # Ensure JSON is valid
    data = json.loads(cleaned_json)

    # Create Excel file
    excel_filename = "extracted_tables2.xlsx"
    with pd.ExcelWriter(excel_filename, engine="openpyxl") as writer:
        for table in data.get("tables", []):
            df = pd.DataFrame(table["rows"], columns=table["headers"])
            sheet_name = f"Table_{table['table_id']}"
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"✅ Excel file '{excel_filename}' created successfully!")

except json.JSONDecodeError as e:
    print(f"❌ JSON Decode Error: {e}")
    print("🔴 Raw LLM Response:", llm_response[:1000])  # Print only the first 1000 chars for debugging
except Exception as e:
    print(f"❌ Unexpected error: {e}")
